<h1>Experiments with built-in and IP cameras on jypyter notebooks</h1>

<p>I'm running these examples on MacBook Pro with Mac OS X 10.11.1</p>
<p>First of all you need to have opencv3 installed. I had best experience when installing it with conda:</p>
<pre><code>conda install -c https://conda.anaconda.org/menpo opencv3</code></pre>
<p>Then for IP camera image frames I have used Opticam i4 and i5 models which are equivalent to Foscam model FI9821W. Foscam Python library (https://github.com/quatanium/foscam-python-lib) is recommended to control IP camera.</p>

<h2>Test built-in camera</h2>
<p>Code retrieved and modified from: https://github.com/bikz05/ipython-notebooks/blob/master/computer-vision/displaying-video-in-ipython-notebook.ipynb</p>
<p>Use jupyter menu Kernel -> Interrupt to stop showing video.</p>

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import cv2
import time
from matplotlib import pyplot as plt
from IPython.display import HTML, clear_output

vid = cv2.VideoCapture(0)
# In case first frames of the video are blank small delay is established here.
time.sleep(0.25)
msg = ""
try:
    while True:
        # Capture frame-by-frame
        ret, frame = vid.read()
        if not ret:
            # Release the Video Device if ret is false
            vid.release()
            # Message to be displayed after releasing the device
            msg = "<p>Frame could not be retrieved! Releasing video resource...</p><img src=\"start.png\" />"
            break
        # Convert the image from OpenCV BGR format to matplotlib RGB format to display the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Turn off the axis
        plt.axis('off')
        # Display the frame
        plt.imshow(frame)
        plt.show()
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device
    msg = "<p>Keyboard interrupt! Releasing video resource...</p>"

HTML("%s<img src=\"start.png\" style=\"width: 320px; height: auto\" />" % msg)

<h2>Test image size reduction and image element extraction and labeling</h2>
<p>You can test different test cases by changing the value of case between 1, 2, 3 and 4.</p>

In [3]:
%pylab inline
import cv2
import time
from IPython.display import HTML, clear_output
# import test1 functions
from test1 import process, labels_to_bboxes, show_plots, largest_labels

# resize image frame by reduce factor
reduce = 50
# http://matplotlib.org/examples/images_contours_and_fields/interpolation_methods.html
interpolation = 'bicubic'
mode = None

camera = cv2.VideoCapture(0)
time.sleep(0.25)

# options are 1, 2, 3 and 4
case = 1

try:
    # loop over the frames of the video
    while True:
        _, frame = camera.read()
        # if the frame could not be grabbed, then we have reached the end of the video
        if not _:
            # Release the Video Device if ret is false
            camera.release()
            # Message to be displayed after releasing the device
            msg = "<p>Frame could not be retrieved! Releasing video resource...</p><img src=\"start2.png\" />"

        if case == 2:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        labels, images = process(frame, 6, reduce, interpolation, mode)
        
        if case == 1:
            labels, images = process(frame, 6, reduce, interpolation, mode)
            bboxes = labels_to_bboxes(labels, images, 500)
            show_plots(images, cols=3, x=16, y=7, bboxes=bboxes)
        elif case == 2:
            labels, images = process(frame, 2, reduce, interpolation, mode)
            rois = largest_labels(labels, images, 1, 500)
            show_plots(rois, cols=3, x=10, y=14)
        elif case == 3:
            labels, images = process(frame, 6, reduce, interpolation, mode)
            show_plots(images, cols=3, x=16, y=10)
        elif case == 4:
            labels, images = process(frame, 6, reduce, interpolation, mode)
            show_plots([im[0] for im, mask in labels], cols=3, x=16, y=10)
        
        clear_output(wait=True)
except KeyboardInterrupt:
    # Release the Video Device (menu: Kernel->Interrupt)
    camera.release()
    clear_output(wait=True)
    # Message to be displayed after releasing the device
    msg = "<p>Keyboard interrupt! Releasing video resource...</p>"

HTML("%s<img src=\"start21.png\" style=\"width: 600px; height: auto\" />\
     <img src=\"start22.png\" style=\"width: 600px; height: auto\" />\
     <img src=\"start23.png\" style=\"width: 600px; height: auto\" />" % msg)

<h2>Test Foscam library and camera control</h2>
<p>Running this example should pan camera up for a second and down for a second.</p>
<p>https://github.com/quatanium/foscam-python-lib is used to control the IP camera</p>

In [4]:
from foscam import FoscamCamera
from time import sleep

ip1 = '192.168.0.16'
ip2 = '192.168.0.18'
port = 88
# needs to have at least operator priviledges for these controls
with open('.operator') as lines:
    username, password = lines.readlines()[0].strip().split(':')

mycam = FoscamCamera(ip2, port, username, password, daemon=True, verbose=False)
mycam.ptz_move_up()
sleep(1)
mycam.ptz_move_down()
sleep(1)
mycam.ptz_stop_run()

<p>Running this example should return appropriate infromation from IP camera</p>

In [5]:
from foscam import FoscamCamera, FOSCAM_SUCCESS

def print_info(returncode, params):
    if returncode != FOSCAM_SUCCESS:
        print ('Failed to get camera info! Error code: %s' % returncode)
    else:
        print (params)

# needs to have at least admin priviledges for these controls
with open('.administrator') as lines:
    username, password = lines.readlines()[0].strip().split(':')

mycam = FoscamCamera(ip2, port, username, password, daemon=False, verbose=False)

In [6]:
a = mycam.get_ip_info(print_info)

{'dns2': '0.0.0.0', 'gate': '0.0.0.0', 'mask': '255.255.255.0', 'dns1': '0.0.0.0', 'ip': '192.168.0.18', 'isDHCP': '1'}


In [7]:
b = mycam.get_port_info(print_info)

{'mediaPort': '88', 'webPort': '88', 'httpsPort': '443'}


In [8]:
c = mycam.refresh_wifi_list(print_info)

{}


<h2>Setting up classes for different cameras</h2>

In [9]:
%pylab inline
# import test2 functions
from test2 import Camera, IPCamera

Populating the interactive namespace from numpy and matplotlib


<h2>Connecting to three different cameras and getting frame height and width</h2>

In [10]:
# visitor priviledges are enough for the rest of the examples
with open('.visitor') as lines:
    username, password = lines.readlines()[0].strip().split(':')

cam = Camera(0)
print (cam.get_frame().shape)
cam.release()
ipcam = IPCamera(ip1, port, username, password)
frame = ipcam.get_frame()
print (frame.shape if frame is not None else None)
ipcam.release()
ipcam2 = IPCamera(ip2, port, username, password)
frame = ipcam2.get_frame()
print (frame.shape if frame is not None else None)
ipcam2.release()

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


<h2>Displaying multiple cameras</h2>
<p>Cameras are also attached with face detection in this example!</p>
<p>For some reason keyboard interrupt has to be done several times at times...</p>

In [11]:
# import test2 functions
from test2 import process, show_frames, clear_output, HTML

cameras = []
cameraA = Camera(0)
cameras.append(cameraA)
cameraB = IPCamera(ip1, port, username, password)
cameras.append(cameraB)
cameraC = IPCamera(ip2, port, username, password)
cameras.append(cameraC)

msg = ""
try:
    cols = len(cameras)
    while True:
        frames = []
        for cam in cameras:
            frame = cam.get_frame(process)
            if frame is not None:
                frames.append(frame)
        if len(frames) > 0:
            show_frames(frames, cols=cols, x=16)

except KeyboardInterrupt:
    # Release the Video Device (menu: Kernel->Interrupt)
    for cam in cameras:
        cam.release()
    # Message to be displayed after releasing the device
    clear_output(wait=True)
    msg = "Releasing video resources..."

HTML("%s<img src=\"start3.png\" style=\"width: 600px; height: auto\" />" % msg)

<h2>Constant IP camera video stream</h2>
<p>I've been unsuccesfull to get a stream from foscam cgi interface so far. Neither MJPEG from http protocol or rtsp protocol has been working dispite of using code examples from Foscam manual and several code examples from Internet.</p> 

In [12]:
try:
    from urllib import urlopen
except ImportError:
    from urllib.request import urlopen

ip = ip1
response = urlopen('http://%s:%s/cgi-bin/CGIStream.cgi?usr=%s&pwd=%s&cmd=GetMJStream' % (ip, 88, username, password))
print(response.read())

b''


In [13]:
ip = ip1
cam = cv2.VideoCapture('rtsp://%s:%s@%s:%s/videoMain' % (username, password, ip, port))
grabbed, frame = cam.read()
print (grabbed, frame)

False None


<h1>The MIT License (MIT)</h1>

<p>Copyright (c) 2016 Marko Manninen</p>